## 迁移学习

训练参数设置

In [2]:
#pre_features_files = {'ResNet50':'ResNet50_pre_out.h5', 'Xception':'Xception_pre_out.h5', 'InceptionV3':'InceptionV3_pre_out.h5'}
pre_features_files = {'ResNet50':'ResNet50_pre_out.h5', 
                      'Xception':'Xception_pre_out.h5', 
                      'InceptionResNetV2':'InceptionResNetV2_pre_out.h5'}

composite_model = True
single_model = 'Xception'

# check training parameter
if not composite_model and single_model not in pre_features_files:
    print("%s not found in pre-trained models" %single_model)
else:
    if composite_model:
        print("will use composite model")
    else:
        print("will use %s model" %single_model)
        
if composite_model:
    model_name = 'Composite_Model'
else:
    model_name = single_model
    
print('model name:', model_name)

will use composite model
model name: Composite_Model


### Tools

In [3]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def show_history(hist, title_name):
    # epoch as x-axis 
    epochs = range(len(hist['acc']))
    
    fig, ax1 = plt.subplots()
    
    ## left scale: acc, val_acc
    # acc
    line_acc = ax1.plot(epochs, hist['acc'], 'r', label='acc')
    # loss
#     ax1.plot(epochs, hist['loss'], 'g', label='loss')
    # val_acc
    line_val_acc = ax1.plot(epochs, hist['val_acc'], 'b', label='val_acc')
    # val_loss
#     ax1.plot(epochs, hist['val_loss'], 'k', label='val_loss')
    
    ax1.set(xlabel='epochs', ylabel='acc',title=title_name)
    
    ## right scale: loss, val_loss
    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis
    color = 'tab:blue'
    ax2.set_ylabel('loss', color=color)  # we already handled the x-label with ax1
    line_loss = ax2.plot(epochs, hist['loss'], 'g', label='loss')
    line_val_loss = ax2.plot(epochs, hist['val_loss'], 'k', label='val_loss')
    ax2.tick_params(axis='y', labelcolor=color)

#     ax1.legend(loc='upper right', fancybox=True,)
#     ax2.legend(loc='lower right', fancybox=True)
    
    ax1.grid(True)
    fig.tight_layout()  # otherwise the right y-label is slightly clipped
    
    acc_patch = mpatches.Patch(color='r', label='acc',linestyle='-')
    val_acc_patch = mpatches.Patch(color='b', label='val-acc')
    loss_patch = mpatches.Patch(color='g', label='loss')
    val_loss_patch = mpatches.Patch(color='k', label='val-loss')
    fig.legend(handles=[acc_patch, val_acc_patch, loss_patch, val_loss_patch],
               ncol=4, loc='lower center',
               mode="expand", borderaxespad=0.,)

    fig.show()

### 载入数据

In [4]:
import h5py
import numpy as np
from sklearn.utils import shuffle
np.random.seed(66)

train_data = []
test_data = []

# #------- single pre-trained model
if not composite_model:
    with h5py.File(pre_features_files[single_model] , 'r') as h:
        train_data.append(np.array(h['train']))
        train_labels = np.array(h['label'])
        test_data.append(np.array(h['test']))
else:#-------- composite model
    for modelname in pre_features_files:
        with h5py.File(pre_features_files[modelname]) as h:
            train_data.append(np.array(h['train']))
            train_labels = np.array(h['label'])
            test_data.append(np.array(h['test']))

train_data = np.concatenate(train_data, axis=1)
test_data = np.concatenate(test_data, axis=1)

print("train shape:",train_data.shape)
print("train_data:",train_data[:5])
# 预存的X_train, y_train是按顺序存放的，前12500是猫，后12500是狗, 这里打乱顺序，使之随机存放
# Note: 打乱的是存放存放顺序，并不改变 X_train , y_train的对应关系
train_data, train_labels = shuffle(train_data, train_labels)

train_data_unclean = train_data
train_labels_unclean = train_labels

train shape: (25000, 5632)
train_data: [[0.06419943 0.5325118  0.61818767 ... 1.4656898  0.10975234 0.13627772]
 [0.01955884 0.2614151  0.00553378 ... 0.01356897 0.04284026 0.        ]
 [0.0658308  0.13139124 0.12781699 ... 0.21625854 0.13916504 0.30605924]
 [0.25487077 0.24022049 0.15445104 ... 0.0120421  0.93518245 0.22036156]
 [0.03490764 0.2764068  0.12088037 ... 0.30616122 0.758728   0.        ]]


#### 剔除train中的异常值

In [5]:
## outliers list
train_outliers=['cat.10636.jpg', 'dog.10161.jpg', 'dog.2614.jpg', 'cat.7564.jpg', 'cat.7377.jpg', 'cat.3216.jpg', 'dog.2422.jpg', 'dog.11266.jpg', 'cat.5071.jpg', 'cat.6345.jpg', 'dog.1773.jpg', 'cat.8456.jpg', 'dog.10190.jpg', 'cat.8921.jpg', 'cat.10029.jpg', 'cat.3004.jpg', 'cat.9171.jpg', 'dog.4218.jpg', 'cat.5418.jpg', 'cat.5974.jpg', 'dog.1895.jpg', 'cat.2520.jpg', 'cat.9983.jpg', 'dog.4507.jpg', 'cat.4338.jpg', 'dog.9188.jpg', 'dog.9517.jpg', 'cat.4308.jpg', 'dog.6475.jpg', 'cat.10365.jpg', 'dog.10237.jpg', 'cat.8470.jpg', 'dog.1625.jpg', 'dog.4367.jpg', 'dog.11299.jpg', 'cat.10536.jpg', 'cat.5351.jpg', 'dog.5604.jpg', 'dog.10747.jpg', 'cat.7464.jpg', 'dog.1259.jpg', 'cat.4688.jpg', 'cat.12272.jpg', 'dog.3889.jpg', 'dog.8898.jpg', 'dog.7706.jpg', 'cat.10712.jpg', 'cat.11184.jpg', 'cat.12476.jpg', 'cat.7968.jpg', 'dog.8736.jpg', 'dog.6725.jpg', 'cat.1139.jpg', 'cat.6348.jpg', 'dog.12376.jpg', 'dog.2339.jpg', 'dog.11437.jpg', 'dog.10801.jpg', 'cat.12424.jpg', 'dog.1194.jpg', 'dog.7076.jpg', 'cat.11039.jpg', 'cat.3672.jpg', 'cat.2939.jpg']

train_outliers.remove('cat.1139.jpg')
train_outliers.remove('cat.3004.jpg')
train_outliers.remove('dog.7706.jpg')

print(len(train_outliers))

61


In [6]:
## create train_generator
# import pandas as pd
from keras.preprocessing.image import *

gen = ImageDataGenerator()
train_generator = gen.flow_from_directory("pre-train", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

Using TensorFlow backend.


Found 25000 images belonging to 2 classes.


In [7]:

outliers_index = [i for i,fn in enumerate(train_generator.filenames) if fn.split('/')[1] in train_outliers]
train_data = np.array([x for i,x in enumerate(train_data_unclean) if i not in outliers_index])
train_data.shape

(24939, 5632)

In [8]:
train_labels = np.array([x for i,x in enumerate(train_labels_unclean) if i not in outliers_index])
train_labels.shape

(24939,)

### 构建模型

In [9]:
from keras.models import Sequential
from keras.models import Model
from keras.layers import GlobalAveragePooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, Input
import numpy as np

np.random.seed(666)

input_tensor = Input(train_data.shape[1:])
x = Dropout(0.5)(input_tensor)
x = Dense(1, activation='sigmoid')(x)
model = Model(input_tensor, x)

model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])

### 在pre-train features上训练模型

In [10]:
from keras.callbacks import EarlyStopping

fit_callback = [EarlyStopping(monitor='val_loss',patience=8)]

model_hist = model.fit(train_data, train_labels,
                       batch_size=128,epochs=50,
                       validation_split = 0.2,
                       verbose=1,
                       callbacks=fit_callback
          )

# show_history(model_hist.history, model_name)

Train on 19951 samples, validate on 4988 samples
Epoch 1/50
19951/19951 [==============================] - 5s 260us/step - loss: 0.0638 - acc: 0.9806 - val_loss: 0.0209 - val_acc: 0.9940
Epoch 2/50
19951/19951 [==============================] - 5s 243us/step - loss: 0.0182 - acc: 0.9944 - val_loss: 0.0157 - val_acc: 0.9950
Epoch 3/50
19951/19951 [==============================] - 5s 237us/step - loss: 0.0144 - acc: 0.9952 - val_loss: 0.0158 - val_acc: 0.9960
Epoch 4/50
19951/19951 [==============================] - 5s 260us/step - loss: 0.0134 - acc: 0.9963 - val_loss: 0.0153 - val_acc: 0.9954
Epoch 5/50
19951/19951 [==============================] - 5s 261us/step - loss: 0.0127 - acc: 0.9963 - val_loss: 0.0154 - val_acc: 0.9952
Epoch 6/50
19951/19951 [==============================] - 7s 348us/step - loss: 0.0112 - acc: 0.9968 - val_loss: 0.0171 - val_acc: 0.9948
Epoch 7/50
19951/19951 [==============================] - 10s 498us/step - loss: 0.0115 - acc: 0.9966 - val_loss: 0.0155 - 

In [11]:
logpath = 'log/'+model_name
    
model.save(logpath+'.h5')
with open(logpath+'_hist.txt','w') as f:
    f.write(str(model_hist.history))

In [12]:
## 查看 history
# Pretty display for notebooks
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches


def read_history_from_file(filename):
    f = open(filename,'r')
    rdbuf = f.read()
    hist = eval(rdbuf)
    f.close()
    
    return hist
    
histfile = 'log/'+model_name+'_hist.txt'
hist = read_history_from_file(histfile)
print(hist)

# title_name = model_name
# show_history(hist, title_name)

{'val_loss': [0.02090158480060464, 0.015739635599862245, 0.01577814605046712, 0.015278635505861704, 0.01541077817581839, 0.017059143748412203, 0.015522938660841134, 0.016660122021602553, 0.016118541233683694, 0.01580147857799433, 0.017058762688855936, 0.01656070227847519], 'val_acc': [0.9939855653568564, 0.9949879711307137, 0.9959903769045709, 0.9953889334402566, 0.9951884522854851, 0.9947874899759422, 0.9951884522854851, 0.9947874899759422, 0.9949879711307137, 0.9947874899759422, 0.9949879711307137, 0.9943865276663993], 'loss': [0.06383925110018049, 0.01824372701330676, 0.014378739819825959, 0.01339605523489106, 0.012686219220756773, 0.01116332047760881, 0.01147974113538405, 0.010360054647303794, 0.00980828053980687, 0.009681007056197115, 0.00885186465080487, 0.008853173605430704], 'acc': [0.980552353295376, 0.9944363691043056, 0.9952383339180993, 0.9963410355669413, 0.9962909127362037, 0.9968422635755625, 0.9965916496011274, 0.9966417723422385, 0.9968923863465491, 0.9971931231815973,

### 用拟合后的模型预测

In [13]:
y_pred = model.predict(test_data, verbose=1)

12500/12500 [==============================] - 1s 67us/step


预测结果写入kaggle sample_submission.csv

In [14]:
y_pred = y_pred.clip(min=0.005, max=0.995)

import pandas as pd
from keras.preprocessing.image import *

df = pd.read_csv("sample_submission.csv")

gen = ImageDataGenerator()
test_generator = gen.flow_from_directory("pre-test", (224, 224), shuffle=False, 
                                         batch_size=16, class_mode=None)

for i, fname in enumerate(test_generator.filenames):
    index = int(fname[fname.rfind('/')+1:fname.rfind('.')])
    df.set_value(index-1, 'label', y_pred[i])

df.to_csv(model_name+'_pred.csv', index=None)
df.head(10)

Found 12500 images belonging to 1 classes.


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


,id,label
0,1,0.995
1,2,0.995
2,3,0.995
3,4,0.995
4,5,0.005
5,6,0.005
6,7,0.005
7,8,0.005
8,9,0.005
9,10,0.005


### 模型可视化

In [3]:
from graphviz import Digraph

dot = Digraph(comment='The Model Chart')

dot.attr('node',shape='record')

dot.node('IN', 'Input|images')
dot.node('PRE-R', 'ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}')
dot.node('PRE-X', 'Xception|{input:|output:}|{(299, 299, 3)|(2048)}')
dot.node('PRE-I', 'InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}')
dot.edge('IN', 'PRE-R')
dot.edge('IN', 'PRE-X')
dot.edge('IN', 'PRE-I')
# dot.edges(['AB', 'AL'])
# dot.edge('PRE-R', 'PRE-X', constraint='false')

dot.node('L3', 'Merge|{input:|output:}|{(2048,3)|2048*3=6144}')
dot.edge('PRE-R', 'L3')
dot.edge('PRE-X', 'L3')
dot.edge('PRE-I', 'L3')

dot.node('L4', 'Dropout|Rate:|0.5')
dot.node('L5', 'Output|{input:|output:}|{6144|1}')
dot.edge('L3', 'L4')
dot.edge('L4', 'L5')

print(dot.source)
dot.render('model-table.gv', view=True)

// The Model Chart
digraph {
	node [shape=record]
	IN [label="Input|images"]
	"PRE-R" [label="ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}"]
	"PRE-X" [label="Xception|{input:|output:}|{(299, 299, 3)|(2048)}"]
	"PRE-I" [label="InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}"]
	IN -> "PRE-R"
	IN -> "PRE-X"
	IN -> "PRE-I"
	L3 [label="Merge|{input:|output:}|{(2048,3)|2048*3=6144}"]
	"PRE-R" -> L3
	"PRE-X" -> L3
	"PRE-I" -> L3
	L4 [label="Dropout|Rate:|0.5"]
	L5 [label="Output|{input:|output:}|{6144|1}"]
	L3 -> L4
	L4 -> L5
}


'model-table.gv.pdf'

####  单模型结构

In [40]:
from graphviz import Digraph

dot = Digraph(comment='The Model Chart')

dot.attr('node',shape='record')

dot.node('IN', 'Input|images')
dot.node('PRE-R', 'ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}')
#dot.node('PRE-X', 'Xception|{input:|output:}|{(299, 299, 3)|(2048)}')
#dot.node('PRE-I', 'InceptionV3|{input:|output:}|{(299, 299, 3)|(2048)}')
dot.edge('IN', 'PRE-R')
#dot.edge('IN', 'PRE-X')
# dot.edge('IN', 'PRE-I')

dot.node('L3', 'Input|{input:|output:}|{(2048,1)|2048}')
dot.edge('PRE-R', 'L3')
# dot.edge('PRE-X', 'L3')
# dot.edge('PRE-I', 'L3')

dot.node('L4', 'Dropout|Rate:|0.5')
dot.node('L5', 'Output|{input:|output:}|{2048|1}')
dot.edge('L3', 'L4')
dot.edge('L4', 'L5')

print(dot.source)
dot.render('sigle-model.gv', view=True)

// The Model Chart
digraph {
	node [shape=record]
	IN [label="Input|images"]
	"PRE-R" [label="ResNet50|{input:|output:}|{(224, 224, 3)|(2048)}"]
	IN -> "PRE-R"
	L3 [label="Input|{input:|output:}|{(2048,1)|2048}"]
	"PRE-R" -> L3
	L4 [label="Dropout|Rate:|0.5"]
	L5 [label="Output|{input:|output:}|{2048|1}"]
	L3 -> L4
	L4 -> L5
}


'sigle-model.gv.pdf'